In [11]:
import pandas as pd
import math

In [2]:
data = pd.read_excel('IR_Spring2021_ph12_7k.xlsx')

In [3]:
#class below is for holding single id and single url
class holder:
    def __init__(self,ID = 0,url = None):
        self.ID = ID
        self.url = url
    def __eq__(self,obj):
        return str(self.ID)+self.url == str(obj.ID)+obj.url
    def __str__(self):
        return str(self.ID)+' '+self.url


In [4]:
#class below is for holding term frequency and list of holder classess
class termValue:
    def __init__(self):
        self.frequency = 0
        self.holderList = []
    def addHolder(self,newHolder):
        if newHolder not in self.holderList:
            self.holderList.append(newHolder)

In [5]:
#function below is for calculating frequency of each term
def frequencyCalc(myDictionary):
    for i in myDictionary:
        myDictionary[i].frequency = len(myDictionary[i].holderList)

In [6]:
#algorithm below is for retrieval of the dictionary
myFile = open('dictionary.txt',encoding = 'UTF-8')
theDictionary = {}
for i in myFile:
    res = i.split('$')
    if res[0] not in theDictionary:
        theDictionary[res[0]] = termValue()
        myHolder = holder(res[1],res[2])
        theDictionary[res[0]].addHolder(myHolder)
    else:
        myHolder = holder(res[1],res[2])
        theDictionary[res[0]].addHolder(myHolder)
myFile.close()

In [7]:
frequencyCalc(theDictionary)

In [44]:
#list below is just for saving list of words
wordsList = []
for i in theDictionary:
    wordsList.append(i)

In [9]:
#normalize 1 method
def normalize1(myList):
    """"
    we should remove some specific characters eg: we do not save dot in the end of sentence with the last word
    we just need to remove if these specific chars are at the last of our word
    """
    for i in range(len(myList)):
        if myList[i].endswith('.'):
            myList[i] = myList[i].replace('.','')
        elif myList[i].endswith('،'):
            myList[i] = myList[i].replace('،','')
        elif myList[i].endswith(':'):
            myList[i] = myList[i].replace(':','')
        else:
            pass
    #the two last members means end of message and it is in all docs so we do not need it
    myList.pop()
    myList.pop()

In [46]:
#algorithm below is saving documents and their bag of words in dictionary docid as key and list of words as value
bagOfWordsDic = {}
for i in range(len(data)):
    docId = data['id'][i]
    listOfwords = data['content'][i].split()
    normalize1(listOfwords)
    bagOfWordsDic[docId] = wordsList

In [47]:
#function below is for calculating tf
def tf_calculator(term,document):
    documentBagOfWords = document.split()
    normalize1(documentBagOfWords)
    n = documentBagOfWords.count(term)
    if n == 0:
        return 0
    return 1+math.log(n)

In [48]:
#function below is for calculating idf
def idf_calculator(term,N,myDictionary):
    return math.log(N/myDictionary[term].frequency)

In [49]:
#function below is for calculating vector of a single document
def createVector(document,listOfWords,myDictionary,N):
    resList = []
    for i in listOfWords:
        resList.append(tf_calculator(i,document)*idf_calculator(i,N,myDictionary))
    return resList

In [37]:
#algorithm below is for calculating all vector spaces
for i in range(len(data)):
    documentId = data['id'][i]
    documentUrl = data['url'][i]
    documentContent = data['content'][i]
    vectorDictionary = {}
    vectorDictionary[str(documentId)+' '+documentUrl] = createVector(documentContent,wordsList,theDictionary,len(data))